In [1]:
# Import python libraries
import numpy as np
import pandas as pd
import importlib
import random
from tqdm import tqdm
import datetime
import geopandas as gpd
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# Set random seed
random.seed(42)
np.random.seed(42)

In [3]:
# import 3 python files that can be reloaded if any changes are made to them
import st_toolkit as geohl
importlib.reload(geohl)

import cri_calc_new as cri
importlib.reload(cri)

import cri_helper as helper
importlib.reload(helper)

<module 'cri_helper' from 'C:\\Users\\kka\\Documents\\intern_run\\cri_helper.py'>

In [4]:
# calculates 5 target factors and the cri for any pair of ships
def calculate_cri(rec_own, rec_target):
    own = rec_own._asdict()
    target = rec_target._asdict()

    ves_dcpa, ves_tcpa, hr, rel_movement_angle, dist_euclid, speed_r = cri.colregs_alarms(own=own, target=target)
    ves_cri = cri.calculate_cri(own, target, ves_dcpa, ves_tcpa, hr, rel_movement_angle, dist_euclid, speed_r)

    return ves_dcpa, ves_tcpa, hr, rel_movement_angle, dist_euclid, speed_r, ves_cri

In [6]:
# calculates the 5 target factors for all ships in the dataset with respect to every other ship
# also calculates the cri between them
def calculate_cri_timeslice(df):
    timeslice_result = []

    for row_i in df.itertuples():
        for row_j in df.itertuples():
            if row_i.Index == row_j.Index:
                continue

            timeslice_result.append([row_i.Index, row_i.mmsi, row_i.geom, row_i.speed, row_i.course,
                                     row_j.Index, row_j.mmsi, row_j.geom, row_j.speed, row_j.course, *calculate_cri(row_i, row_j)])

#     return pd.DataFrame(timeslice_result, columns=['own', 'target', 'dcpa', 'tcpa', 'hr', 'rel_movement_angle', 'dist_euclid', 'speed_r', 'cri'])
    return pd.DataFrame(timeslice_result, columns=['own_Index', 'own_mmsi', 'own_geom', 'own_speed', 'own_course',
                                                   'target_Index', 'target_mmsi', 'target_geom', 'target_speed', 'target_course',
                                                   'dcpa', 'tcpa', 'hr', 'rel_movement_angle', 'dist_euclid', 'speed_r', 'cri'])

In [7]:
# reading the pre-processed pickel file from the pre-processing notebook
gdf_sub_moving = pd.read_pickle('Final\\gdf_sub_moving.pickle')

In [8]:
gdf_sub_moving

,timestamp,mmsi,lon,lat,speed,course,datetime,geom
4184635,1.531402e+09,36549925b2f9c0cf560db9f862e494060730d934a0adf2...,23.611212,37.958570,1.145243,82.771737,2018-07-12 13:34:13,POINT (23.61121 37.95857)
4184638,1.531402e+09,490201f75b5ddc3766b08b79c84ff3ff6f3737c31a8906...,23.562175,37.871142,16.099324,35.240754,2018-07-12 13:34:13,POINT (23.56218 37.87114)
4184639,1.531402e+09,f0d67fcc12c0c3a723b8e06e52122b8d18260ea68aa801...,23.662430,37.923905,13.329369,43.388931,2018-07-12 13:34:13,POINT (23.66243 37.9239)
4184640,1.531402e+09,1eee5599ef5de5c07df6698ab4d4a6e6bbe4a2be252088...,23.510918,37.823275,32.920703,35.389401,2018-07-12 13:34:13,POINT (23.51092 37.82328)
4184641,1.531402e+09,312c18fc5357c78c31068fa01d25cdf0c956a31f361b81...,23.507717,37.822967,4.885682,207.771577,2018-07-12 13:34:13,POINT (23.50772 37.82297)
...,...,...,...,...,...,...,...,...
4736978,1.531440e+09,396a8c899036f5a36aac63841841d4e52b7a21a7adf644...,23.179833,37.873217,11.826265,100.369198,2018-07-12 23:59:27,POINT (23.17983 37.87322)
4737001,1.531440e+09,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,23.683000,37.931000,1.935091,40.246741,2018-07-12 23:59:35,POINT (23.683 37.931)
5153653,1.531429e+09,9853c62eb450dee1bf42639a61e714abcb743428820cf7...,23.595972,37.953402,9.810878,349.933504,2018-07-12 21:00:02,POINT (23.59597 37.9534)
5153661,1.531429e+09,3fc28f4d2b3c7cb5f68d5b71f6fda727e4f8cd66515d98...,23.619285,37.936162,12.291928,13.753747,2018-07-12 21:00:03,POINT (23.61928 37.93616)


Calculate CRI

In [9]:
# using parallel processing to calculate the target factors and cri for all own-ship-target-ship pairs
from joblib import Parallel, delayed
import multiprocessing


def applyParallel(dfGrouped, func, n_jobs=-1):
    n_jobs = multiprocessing.cpu_count() if n_jobs == -1 else n_jobs
    print(f'Scaling {func} to {n_jobs} CPUs')

    retLst = Parallel(n_jobs=n_jobs)(delayed(func)(group) for name, group in tqdm(dfGrouped))
    return pd.concat(retLst)


gdf_vcra = applyParallel(gdf_sub_moving.groupby(['datetime']), calculate_cri_timeslice, 18)

Scaling <function calculate_cri_timeslice at 0x0000022ABC73E520> to 18 CPUs


  0%|                                                                                        | 0/65533 [00:00<?, ?it/s]C:\ProgramData\anaconda3\Lib\site-packages\tqdm\std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.

  for obj in iterable:

100%|███████████████████████████████████████████████████████████████████████████| 65533/65533 [05:32<00:00, 197.23it/s]


In [10]:
gdf_vcra

,own_Index,own_mmsi,own_geom,own_speed,own_course,target_Index,target_mmsi,target_geom,target_speed,target_course,dcpa,tcpa,hr,rel_movement_angle,dist_euclid,speed_r,cri
0,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,10.453580,-0.223472,58.133423,4.937556,10.655910,9.247937,0.000000
1,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,3.108515,-0.005242,-101.762145,6.178911,3.116042,41.291932,0.000000
2,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,10.453580,-0.223472,-58.133423,1.795963,10.655910,9.247937,0.000000
3,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,3.943769,0.247131,-159.895568,0.120345,10.479444,39.287023,0.064588
4,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,3.108515,-0.005242,101.762145,3.037318,3.116042,41.291932,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,4736527,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.583933333333302 37.95276666666671),1.657704,188.961942,4736526,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,-1.683891,5.407627,30.636575,4.709079,6.178706,1.099340,0.192224
0,4736739,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,4736740,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.5839166666667 37.9529166666667),1.521905,30.599527,-5.805277,0.584286,-188.998989,0.625693,6.182760,3.640803,0.023216
1,4736740,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.5839166666667 37.9529166666667),1.521905,30.599527,4736739,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,-5.805277,0.584286,188.998989,3.767286,6.182760,3.640803,0.035265
0,4736779,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.58386666666669 37.9528666666667),1.251172,218.256384,4736780,396a8c899036f5a36aac63841841d4e52b7a21a7adf644...,POINT (23.176383333333305 37.8736666666667),11.578077,100.759199,8.371487,1.949450,-117.497185,1.667531,25.225222,12.206309,0.072288


In [11]:
# merge the dfs based on own index
gdf_vcra_mi = gdf_vcra.copy()
gdf_vcra_mi = pd.merge(gdf_vcra_mi, gdf_sub_moving.datetime, left_on='own_Index', right_index=True)
gdf_vcra_mi.set_index(['datetime'], inplace=True)

In [12]:
gdf_vcra_mi

,own_Index,own_mmsi,own_geom,own_speed,own_course,target_Index,target_mmsi,target_geom,target_speed,target_course,dcpa,tcpa,hr,rel_movement_angle,dist_euclid,speed_r,cri
datetime,,,,,,,,,,,,,,,,,
2018-07-12 00:00:41,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,10.453580,-0.223472,58.133423,4.937556,10.655910,9.247937,0.000000
2018-07-12 00:00:41,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,3.108515,-0.005242,-101.762145,6.178911,3.116042,41.291932,0.000000
2018-07-12 00:00:41,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,10.453580,-0.223472,-58.133423,1.795963,10.655910,9.247937,0.000000
2018-07-12 00:00:41,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,3.943769,0.247131,-159.895568,0.120345,10.479444,39.287023,0.064588
2018-07-12 00:00:41,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,3.108515,-0.005242,101.762145,3.037318,3.116042,41.291932,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-12 23:57:25,4736527,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.583933333333302 37.95276666666671),1.657704,188.961942,4736526,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,-1.683891,5.407627,30.636575,4.709079,6.178706,1.099340,0.192224
2018-07-12 23:58:25,4736739,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,4736740,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.5839166666667 37.9529166666667),1.521905,30.599527,-5.805277,0.584286,-188.998989,0.625693,6.182760,3.640803,0.023216
2018-07-12 23:58:25,4736740,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.5839166666667 37.9529166666667),1.521905,30.599527,4736739,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,-5.805277,0.584286,188.998989,3.767286,6.182760,3.640803,0.035265


In [13]:
date_times = gdf_vcra_mi.index

# Convert to Unix timestamp
gdf_vcra_mi['timestamp_'] = (pd.to_datetime(date_times) - pd.Timestamp("1970-01-01")) / pd.Timedelta('1s')

gdf_vcra_mi

,own_Index,own_mmsi,own_geom,own_speed,own_course,target_Index,target_mmsi,target_geom,target_speed,target_course,dcpa,tcpa,hr,rel_movement_angle,dist_euclid,speed_r,cri,timestamp_
datetime,,,,,,,,,,,,,,,,,,
2018-07-12 00:00:41,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,10.453580,-0.223472,58.133423,4.937556,10.655910,9.247937,0.000000,1.531354e+09
2018-07-12 00:00:41,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,3.108515,-0.005242,-101.762145,6.178911,3.116042,41.291932,0.000000,1.531354e+09
2018-07-12 00:00:41,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,10.453580,-0.223472,-58.133423,1.795963,10.655910,9.247937,0.000000,1.531354e+09
2018-07-12 00:00:41,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,3.943769,0.247131,-159.895568,0.120345,10.479444,39.287023,0.064588,1.531354e+09
2018-07-12 00:00:41,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,3.108515,-0.005242,101.762145,3.037318,3.116042,41.291932,0.000000,1.531354e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-12 23:57:25,4736527,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.583933333333302 37.95276666666671),1.657704,188.961942,4736526,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,-1.683891,5.407627,30.636575,4.709079,6.178706,1.099340,0.192224,1.531440e+09
2018-07-12 23:58:25,4736739,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,4736740,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.5839166666667 37.9529166666667),1.521905,30.599527,-5.805277,0.584286,-188.998989,0.625693,6.182760,3.640803,0.023216,1.531440e+09
2018-07-12 23:58:25,4736740,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.5839166666667 37.9529166666667),1.521905,30.599527,4736739,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,-5.805277,0.584286,188.998989,3.767286,6.182760,3.640803,0.035265,1.531440e+09


In [14]:
saint_input = gdf_vcra_mi.loc[gdf_vcra_mi.own_Index.isin(gdf_sub_moving.index.values)].copy()

In [15]:
saint_input

,own_Index,own_mmsi,own_geom,own_speed,own_course,target_Index,target_mmsi,target_geom,target_speed,target_course,dcpa,tcpa,hr,rel_movement_angle,dist_euclid,speed_r,cri,timestamp_
datetime,,,,,,,,,,,,,,,,,,
2018-07-12 00:00:41,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,10.453580,-0.223472,58.133423,4.937556,10.655910,9.247937,0.000000,1.531354e+09
2018-07-12 00:00:41,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,3.108515,-0.005242,-101.762145,6.178911,3.116042,41.291932,0.000000,1.531354e+09
2018-07-12 00:00:41,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,10.453580,-0.223472,-58.133423,1.795963,10.655910,9.247937,0.000000,1.531354e+09
2018-07-12 00:00:41,4237953,1cf96efc158dc90e0bdb4f9577cb9946e1b452a83beb1e...,POINT (23.551705 37.9538083333333),1.206454,167.395637,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,3.943769,0.247131,-159.895568,0.120345,10.479444,39.287023,0.064588,1.531354e+09
2018-07-12 00:00:41,4237957,7c69ef3e802069386e5e4bc104751558f44be8e2f0cd0e...,POINT (23.59745 37.8210333333333),38.151892,7.500070,4237952,5afbca147fdf95afc902fc8a1faf0d40e861c5cbff7a5b...,POINT (23.54639 37.8139516666667),9.827940,109.262214,3.108515,-0.005242,101.762145,3.037318,3.116042,41.291932,0.000000,1.531354e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-12 23:57:25,4736527,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.583933333333302 37.95276666666671),1.657704,188.961942,4736526,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,-1.683891,5.407627,30.636575,4.709079,6.178706,1.099340,0.192224,1.531440e+09
2018-07-12 23:58:25,4736739,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,4736740,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.5839166666667 37.9529166666667),1.521905,30.599527,-5.805277,0.584286,-188.998989,0.625693,6.182760,3.640803,0.023216,1.531440e+09
2018-07-12 23:58:25,4736740,8e75c1624d2d78c0f3df3598aaa14f5fcfe6252557fb7b...,POINT (23.5839166666667 37.9529166666667),1.521905,30.599527,4736739,19b6ca4e78cdbe685310599b3934c228e81b5cb6bbaf6a...,POINT (23.6829283333333 37.9309316666667),2.129840,219.598516,-5.805277,0.584286,188.998989,3.767286,6.182760,3.640803,0.035265,1.531440e+09


In [16]:
saint_input.to_pickle('Final\\saint_input.pickle')

In [20]:
# input and ground truth values of model
X = saint_input[['dist_euclid', 'own_speed', 'target_speed', 'own_course', 'target_course']].values
y = saint_input[['cri']].values.ravel()

In [23]:
# convert to pickel file
X_df = pd.DataFrame(X, columns=['dist_euclid', 'own_speed', 'target_speed', 'own_course', 'target_course'])
X_df.to_pickle('Final\\X_df.pkl')

In [24]:
# convert ground truths to pickel file
y_cri_df = pd.DataFrame(y,columns=['cri'])
y_cri_df.to_pickle('Final\\y_cri_df.pkl')